## Install cobrapy

In [8]:
%pip install cobra --quiet

## Import CAM model from Github
Importing pH updating functions and excel from github

In [9]:
# Retrieving the model
!wget -q https://raw.githubusercontent.com/sshameer/DielLeafModeling/refs/heads/main/C3_model.sbml
!wget -q https://raw.githubusercontent.com/sshameer/DielLeafModeling/refs/heads/main/Functions.py
!wget -q https://raw.githubusercontent.com/sshameer/DielLeafModeling/refs/heads/main/MetaboliteChargedStates.xls

## Import functions required to read sbml, write sbml and perform pFBA

In [10]:
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model
from cobra.flux_analysis import pfba

## Import the model and constrain diel leaf model for CAM photosynthesis

In [11]:
C3_model = read_sbml_model("C3_model.sbml")
CAM_model = C3_model.copy()

In [12]:
# restricting CO2 uptake to night
CAM_model.reactions.CO2_tx1.upper_bound = 0
CAM_model.reactions.CO2_tx1.lower_bound = 0


# setting photon uptake
PPFD = 800
CAM_model.reactions.Photon_tx1.upper_bound = PPFD
CAM_model.reactions.Photon_tx1.lower_bound = 0

# setting the maintenance cost
VATPase = 0.0049*PPFD+2.7851
CAM_model.reactions.get_by_id("ATPase_tx1").bounds= (VATPase,VATPase)
solution = pfba(CAM_model)

In [13]:
CAM_model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e2,CO2_tx2,42.52,1,100.00%
WATER_e1,H2O_tx1,19.11,0,0.00%
WATER_e2,H2O_tx2,20.01,0,0.00%
NITRATE_e1,Nitrate_tx1,0.2773,0,0.00%
NITRATE_e2,Nitrate_tx2,0.1848,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,6.115,0,0.00%
Photon_e1,Photon_tx1,800,0,0.00%
SULFATE_e1,SO4_tx1,0.003533,0,0.00%
PROTON_c1,unlProtHYPO_c1,0.4024,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux


### Note that the CO2 is taken up via CO2_tx2, where the 2 tag stands for night-time.

##Accounting to the acidification of vacuole at night in CAM leaves
Setting the model night vacuolar pH from 5.5 to 3.3

In [14]:
from Functions import convertToClassicalModel, convertToFractionalCharges
converted_model = convertToClassicalModel(CAM_model, comp="v2",updateCharges="")
CAM_model = convertToFractionalCharges(converted_model, infile=r"MetaboliteChargedStates.xls", compH={"v2":3.3},TransferTag="")

Verifying the updated pH in both day and night vacuole with Malate

In [15]:
rxn = CAM_model.reactions.get_by_id("MAL_v_dielTransfer")

for met in rxn.metabolites:
  print(met.id+"\t\t"+str(rxn.metabolites[met]))

aMAL_v1		-0.3
MAL_v1		-0.7
aaMAL_v2		0.438
aMAL_v2		0.554
MAL_v2		0.008
PROTON_v2		-1.13


**Note:** Day time vacuole is at a pH of 5.5 and at this pH malate exists as two charged forms, 70% as malate2- and 30% malate1-. Night time vacuole here is considered to be at a pH of 3.3. Under this pH, malate showing three fractional forms with 0.8% as malate2- , 55.4 as malate1- and 43.8 as malatic acid.

CAM plants are know to show a vacuolar influx of H+ and malate in a 2:1 ratio

In [16]:
# print noctural influx of vacuolar malate
rxn = CAM_model.reactions.get_by_id("MAL_PROTON_vc2")
print(rxn.reaction)
# sum of coefficents of all malate/malic acid species in the reaction = 1
malate_acc = rxn.flux*1
print("Nocturnal influx of malate = "+str(round(malate_acc,2)))

MAL_c2 + 1.43 PROTON_v2 --> 0.008 MAL_v2 + 0.554 aMAL_v2 + 0.438 aaMAL_v2
Nocturnal influx of malate = 48.23


In [18]:
# print the nocturnal influx of H+
rxn = CAM_model.reactions.get_by_id("PROTONATP_rev_vc2")
print(rxn.reaction)
# coefficient of vacuolar H+ in V-ATPase reaction = 2
H_acc = rxn.flux*2
print("Nocturnal influx of H+ = "+str(round(H_acc,2)))

0.65 ATP_c2 + 1.45 PROTON_c2 + WATER_c2 + 0.35 aATP_c2 --> 0.5 ADP_c2 + 2.0 PROTON_v2 + 0.7 Pi_c2 + 0.5 aADP_c2 + 0.3 aPi_c2
Nocturnal influx of H+ = 97.36


In [19]:
print("H+/malate- influx into the vacuole = "+str(round(H_acc/malate_acc,2)))

H+/malate- influx into the vacuole = 2.02
